In [2]:
from socket import socket, AF_INET, SOCK_DGRAM
from struct import *
from datetime import *

def print_bin(label, bin):
	print(label, end=" : ")
	for i in bin:
		print(format(i, "02x"), end=" ")
	print("")

# UDPソケット

SOCKET_BUFFSIZE = 2048
ETHIO_IPADDR = '192.168.1.203'
ETHIO_UDPPORT = 16241
sock = socket(AF_INET, SOCK_DGRAM)

# リクエストパケット

ETHIO_PACKET_FOURCC = b'AVMM'
ETHIO_PACKET_ENDCMD = b'\x7f\x00\xff\xff'

send_bin = ETHIO_PACKET_FOURCC + pack('!BxHL', 0x54, 8, 0x10000000) + ETHIO_PACKET_ENDCMD
print_bin("send packet", send_bin)

# UDP送受信 

sock.sendto(send_bin, (ETHIO_IPADDR, ETHIO_UDPPORT))
res_bin, addr = sock.recvfrom(SOCKET_BUFFSIZE)

# レスポンスパケット

print_bin("recv packet",res_bin)

# PERIDOTのSystem IDペリフェラルの値を取り出す

sysid, timestamp = unpack('LL', res_bin[8:16])
dt = datetime.fromtimestamp(timestamp)
print("System ID =",hex(sysid),", Build",dt)

sock.close()


send packet : 41 56 4d 4d 54 00 00 08 10 00 00 00 7f 00 ff ff 
recv packet : 41 56 4d 4d d4 00 00 08 01 00 c0 72 25 02 2e 63 ff 00 00 00 
System ID = 0x72c00001 , Build 2022-09-24 03:59:49
